In [ ]:
import pandas as pd
from IPython.display import display

In [24]:

# Function to load field mappings
def loadMapping(path, prefix):
    mapping = {}
    with open(path, "r", encoding="utf-8") as file:
        
        for line in file:
            parts = line.strip().split("\t")
            if len(parts) == 2:
                field_id = parts[0]
                field_name = parts[1]
                if field_id.startswith(prefix):  # Ensure correct prefix (S or D)
                    field_id = 'f' + field_id[1:]
                    mapping[field_id] = field_name
    return mapping

# Load static and dynamic field mappings
staticField = loadMapping("staticFields.Txt", "S")
dynamicField = loadMapping("DynamicFields.Txt", "D")

# Check first few values
print("Static Fields:", list(staticField.items())[:5])
print("Dynamic Fields:", list(dynamicField.items())[:5])


Static Fields: [('f1', 'Add (3) / Modify (2) / Delete (1) Flag'), ('f2', 'Root symbol (for options,...)'), ('f3', 'DTB version code'), ('f4', 'Expiration day'), ('f5', 'Expiration month')]
Dynamic Fields: [('f0', 'Symbol name - Not used for Snapshot or Tick Data - please see H fields'), ('f1', 'Time in minutes of the day (0..1440)'), ('f2', 'Last price'), ('f3', 'Last volume'), ('f4', 'Bid price')]


In [ ]:
# Function to parse data.txt and extract relevant fields
def parseData(line, staticField, dynamicField):
    parts = line.strip().split("|")
    
    if len(parts) < 7:
        return None  # Skip lines with no data

    # Extract basic fields
    date = parts[0]
    timestamp = parts[1]
    msgType = parts[2]
    instrument_code = parts[3]

    # Determine if we need static or dynamic field mapping
    fieldMapping = staticField if msgType == "S" else dynamicField

    # Extract field values
    fieldValues = {}
    for field in parts[7:]:
        keyValue = field.split("=")
        if len(keyValue) == 2:
            field_id, value = keyValue
            field_name = fieldMapping.get(field_id, field_id)  # Convert to readable name
            fieldValues[field_name] = value

    return {
        "date": date,
        "timestamp": timestamp,
        "message_type": msgType,
        "instrument": instrument_code,
        **fieldValues
    }



In [ ]:
# Loading the data.txt file as a dataframe
def loadDatFile(path, staticField, dynamicField):
    data_records = []
    
    # Open the file in read mode and run each line to parse data
    with open(path, "r", encoding="utf-8") as file:
        for line in file:
            parsed_data = parseData(line, staticField, dynamicField)
            if parsed_data:
                data_records.append(parsed_data)

    df = pd.DataFrame(data_records)
    df = df.rename(columns={**staticField, **dynamicField}, errors="ignore")
    return df

# Method call on data.txt
dataDf = loadDatFile("data.txt", staticField, dynamicField)
dataDf.fillna("NA", inplace=True)

# Show first 5 rows
display(dataDf.head()) 


,date,timestamp,message_type,instrument,Add (3) / Modify (2) / Delete (1) Flag,"Root symbol (for options,...)",Expiration day,Expiration month,Expiration year (modulo 100),Strike price,...,Julian Date of last Trade (EST) (MCR),Yesterday's close price,Open price,High price,Low price,# of trades since market opening,Cumulative volume,Calculated Turnover,VWAP price (frac),Origination market of last trade for composite updates
0,15-04-22,00:00:08:285,S,IXN22EKB61500,2,IXN,20,5,2022,615,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,15-04-22,00:00:14:401,S,IXN22BIB63500,2,IXN,18,2,2022,635,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,15-04-22,00:00:16:637,S,IXN22EKB63500,2,IXN,20,5,2022,635,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,15-04-22,00:00:23:085,S,IXN22QKB61500,2,IXN,20,5,2022,615,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,15-04-22,00:00:31:035,S,IXN24AJB64500,2,IXN,19,1,2024,645,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
def filter_data(df, instrument_codes, start_time, end_time, requested_fields):
    # Filtering data by instrument codes
    df = df[df["instrument"].isin(instrument_codes)]
    
    # Converting timestamps to datetime objects
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%H:%M:%S:%f")
    start_time = pd.to_datetime(start_time, format="%H:%M:%S:%f")
    end_time = pd.to_datetime(end_time, format="%H:%M:%S:%f")
    
    # Filtering by the specified time range
    df = df[(df["timestamp"] >= start_time) & (df["timestamp"] <= end_time)]
    
    # Determining which requested fields are actually present in the DataFrame
    req_fields_in_df = [field for field in requested_fields if field in df.columns]
    
    # Creating a list of columns to display: timestamp, instrument, plus the requested fields available
    selected_columns = ["timestamp", "instrument"] + req_fields_in_df
    df = df[selected_columns]
    
    # Remove rows where all requested fields are empty 
    if req_fields_in_df:
        df = df[
            ~df[req_fields_in_df].apply(
                lambda row: all(pd.isna(x) or (isinstance(x, str) and x.strip() == "NA") for x in row),
                axis=1
            )
        ]
    
    return df

# Creating user Input fields
instrument_codes_input = input("Enter instrument codes separated by commas: ")
instrument_codes = [code.strip() for code in instrument_codes_input.split(",") if code.strip()]

start_time = input("Enter start timestamp (format HH:MM:SS:fff): ")
end_time = input("Enter end timestamp (format HH:MM:SS:fff): ")

requested_fields_input = input("Enter requested fields separated by commas: ")
requested_fields = [field.strip() for field in requested_fields_input.split(",") if field.strip()]

# Filter the data
filtered_df = filter_data(dataDf, instrument_codes, start_time, end_time, requested_fields)
# Display the output table 
display(filtered_df)


<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 14 to 13505
Data columns (total 5 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   timestamp                               18 non-null     datetime64[ns]
 1   instrument                              18 non-null     object        
 2   Bid market                              18 non-null     object        
 3   Bid size                                18 non-null     object        
 4   Add (3) / Modify (2) / Delete (1) Flag  18 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 864.0+ bytes
None


C:\Users\gupta\AppData\Local\Temp\ipykernel_37564\3307759661.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["timestamp"] = pd.to_datetime(df["timestamp"], format="%H:%M:%S:%f")


,timestamp,instrument,Bid market,Bid size,Add (3) / Modify (2) / Delete (1) Flag
14,1900-01-01 00:00:35.458,IXN22QKB63500,NA,NA,2
4007,1900-01-01 13:30:00.170,IXN23AKB31500,36,1,NA
4021,1900-01-01 13:30:29.665,IXN23AKB31500,2,1,NA
4852,1900-01-01 13:31:27.889,IXN22QKB63500,15,1,NA
5947,1900-01-01 13:38:45.999,IXN22QKB63500,36,12,NA
6862,1900-01-01 13:45:59.575,IXN22QKB63500,2,12,NA
7187,1900-01-01 13:51:16.172,IXN23AKB31500,11,49,NA
7755,1900-01-01 13:53:08.946,IXN22QKB63500,24,13,NA
8231,1900-01-01 13:58:08.886,IXN23AKB31500,36,40,NA
8986,1900-01-01 14:00:07.492,IXN22QKB63500,17,3,NA
